In [26]:
!git clone https://github.com/peppermenta/faceHack

fatal: destination path 'faceHack' already exists and is not an empty directory.


In [27]:
!pip3 install -r faceHack/requirements.txt

In [28]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal',transform=None):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type
    self.transform = transform

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    if(self.transform):
      X = self.transform(X)
    return X,y


In [29]:
faceTransform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.ToTensor(),
])

In [39]:
device = torch.device('cuda')
print(device)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = False
for param in model.layer4.parameters():
  param.requires_grad = True

in_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=in_features,out_features=128),
    torch.nn.Linear(in_features=128,out_features=2)
)

model = model.to(device)

trainDataset = FaceDataset(transform=faceTransform)
testDataset = FaceDataset(train=False,transform=faceTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=True,num_workers=2)
testLoader = torch.utils.data.DataLoader(testDataset,batch_size=32,shuffle=False,num_workers=2)

cuda


In [40]:
model.train()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=0.0001)
epochs = 20
lambda1 = 0.0001

loss_hist = []
train_acc_hist = []
test_acc_hist = []

for epoch in range(epochs):
  tot_loss = 0
  correct = 0
  for i,data in enumerate(trainLoader):
    X,y = data
    X = X.to(device)
    y = y.to(device)

    out = model(X)

    _,pred = torch.max(out,dim=1)
    correct += (pred==y).sum().item()

    loss = criterion(out,y)
    for p in model.parameters():
      if p.requires_grad == True:
        loss += lambda1*torch.sum(p**2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    if(i%20==0):
      print('Epoch:({},{}): Loss={}'.format(epoch,i,loss.item()))

  loss_hist.append(tot_loss)
  print('Train Accuracy is {}'.format(correct/len(trainDataset)))
  train_acc_hist.append(correct/len(trainDataset))

  test_correct = 0
  with torch.no_grad():
    for X,y in testLoader:
      X = X.to(device)
      y = y.to(device)
      out = model(X)
      _,pred = torch.max(out,dim=1)
      test_correct += (pred==y).sum().item()
  print('Test accuracy is {}'.format(test_correct/len(testDataset)))
  test_acc_hist.append(test_correct/len(testDataset))

import matplotlib.pyplot as plt

plt.plot([i for i in range(epochs)],loss_hist)
plt.show()

plt.plot([i for i in range(epochs)],train_acc_hist)
plt.plot([i for i in range(epochs)],test_acc_hist)
plt.legend(['Train Accuracy','Test Accuracy'])
plt.show()

Epoch:(0,0): Loss=1.2052834033966064
Epoch:(0,20): Loss=0.8940039277076721
Epoch:(0,40): Loss=0.9091390371322632
Epoch:(0,60): Loss=0.936480700969696
Epoch:(0,80): Loss=0.9123072028160095
Epoch:(0,100): Loss=0.8361541032791138
Epoch:(0,120): Loss=0.9347668886184692
Epoch:(0,140): Loss=0.8243680596351624
Train Accuracy is 0.7913475470916994
Test accuracy is 0.8100558659217877
Epoch:(1,0): Loss=0.710847795009613
Epoch:(1,20): Loss=0.7991715669631958
Epoch:(1,40): Loss=0.6995925307273865
Epoch:(1,60): Loss=0.8800614476203918
Epoch:(1,80): Loss=0.9180770516395569
Epoch:(1,100): Loss=0.7769805192947388
Epoch:(1,120): Loss=0.6507053971290588
Epoch:(1,140): Loss=0.7632126808166504
Train Accuracy is 0.8712481887807907
Test accuracy is 0.8417132216014898
Epoch:(2,0): Loss=0.7876995205879211
Epoch:(2,20): Loss=0.6300920844078064
Epoch:(2,40): Loss=0.5786898136138916
Epoch:(2,60): Loss=0.6473541259765625
Epoch:(2,80): Loss=0.5775981545448303
Epoch:(2,100): Loss=0.6960486769676208
Epoch:(2,120): L

KeyboardInterrupt: ignored

In [ ]:
model.eval()